In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
import json

# Load the JSON file
with open('intents.json') as file:  #type it whith your folder location of the dataset 
    data = json.load(file)

In [ ]:
tags = []
patterns = []
responses = {}
for intent in data['intents']:
  responses[intent['tag']]= intent['responses']
  for lines in intent['patterns']:
    patterns.append(lines)
    tags.append(intent['tag'])

In [ ]:
data = pd.DataFrame({"patterns": patterns,
                     "tags": tags})

In [ ]:
data

In [ ]:
#Preprocessing data

In [ ]:
import string

In [ ]:
data['patterns'] = data['patterns'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation]) #remove punctuation and lowercase
data['patterns'] = data['patterns'].apply(lambda wrd: ''.join(wrd))
data

In [ ]:
#tokenize the data

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['patterns'])

train = tokenizer.texts_to_sequences(data['patterns'])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

In [ ]:
# encoding the output

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

In [ ]:
# define the vocab
vocabulary = len(tokenizer.word_index)
print("number of unique words", vocabulary)
output_length = le.classes_.shape[0]
print("output length", output_length)

In [ ]:
 # neural network LSTM
from keras.models import Model


i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1, 10)(i)
x = LSTM(10, return_sequences = True)(x)
x = Flatten()(x)
x = Dense(output_length, activation = "softmax")(x)
model = Model(i,x)

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
train = model.fit(x_train, y_train, epochs = 1000)


In [ ]:
# accuracy plotting

In [ ]:
plt.plot(train.history['accuracy'], label = 'training set accuracy')
plt.plot(train.history['loss'], label = 'training set loss')
plt.legend()

In [ ]:
# chatting
import random

In [23]:
while True:
  texts_p = []
  prediction_input = input('You')

  # remove punct and convert to lowrcase4
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)

  # token and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input], input_shape)

  # getting output
  output = model.predict(prediction_input)
  output = output.argmax()

  # finding the right tag and predict
  response_tag = le.inverse_transform([output])[0]
  print("curhat's boot : ", random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break